In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2///')
%cd /home/schirrmr/


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

In [ ]:
# 1d example uniform 2,-2 two points at 1,-1

In [ ]:
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var, var_to_np
from IPython.display import display

In [ ]:
x = np_to_var([-1,1], dtype=np.float32, requires_grad=True)
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


#### now with weight norm regularization on a point at 0,0

In [ ]:
from torch.nn import ConstantPad2d
import torch as th
from torch import nn
class ConcatReLU(nn.Module):
    def __init__(self):
        super(ConcatReLU, self).__init__()

    def forward(self, x):
        return th.cat((nn.functional.relu(x), nn.functional.relu(-x)), dim=1)

class Clip(nn.Module):
    def __init__(self):
        super(Clip, self).__init__()

    def forward(self, x):
        return th.clamp(x, 0,1.2)
    

In [ ]:
class PointWiseMultLayer(nn.Module):
    """Multiplies features (hardcoded to 4 with different "scaling factors")"""
    def __init__(self):
        super(PointWiseMultLayer,self).__init__()
        self.weights = th.ones(4, requires_grad=True) # not a parameter!! will not be returned by model.parameters()
    def forward(self, x):
        return x * self.weights.unsqueeze(0)

valid_x = np_to_var([-0.5, 0.5], dtype=np.float32, requires_grad=True)

from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(0, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,2), ConcatReLU(), 
                      Clip(), PointWiseMultLayer(), nn.Linear(4,1))
model[0].bias.data[0] = -1
model[0].bias.data[1] = 1
model[0].weight.data[0] = 1
model[0].weight.data[1] = 1
model[4].weight.data[:] = 0.25
model[4].bias.data[:] = 0


fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
plt.legend(("Discriminator score",))
plt.ylabel("Discriminator score")
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))
part_model = nn.Sequential(model[0], model[1], model[2])
out_part = part_model(x_fake.unsqueeze(1))

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
plt.legend(("Abs first filter", # Since sum of filter after Concat ReLU
           "Abs second filter",))
display(fig)
plt.close(fig)

optim_wnorm = th.optim.Adam([model[3].weights], lr=5e-3)
optimizer = th.optim.Adam(model.parameters(), lr=5e-3)

In [ ]:
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(x.unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss = -(th.mean(real_out) - th.mean(fake_out))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    valid_out = model(valid_x.unsqueeze(1))
    loss = -th.mean(valid_out)
    optim_wnorm.zero_grad()
    loss.backward()
    optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        display(fig)
        plt.close(fig)
        print("loss {:.3f}".format(loss.item()))
        print("wnorms", [model[3].weights])
        print(model[0].weight)
        print(model[0].bias)
        part_model = nn.Sequential(model[0], model[1], model[2])
        out_part = part_model(x_fake.unsqueeze(1))
        fig = plt.figure(figsize=(12,3))

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
        plt.legend(("Abs first filter", # Since sum of filter after Concat ReLU
                   "Abs second filter",))
        display(fig)
        plt.close(fig)

### optimize generated points

In [ ]:
valid_x = np_to_var([-0.5, 0.5], dtype=np.float32, requires_grad=True)

from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(0, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,2), ConcatReLU(), 
                      Clip(), PointWiseMultLayer(), nn.Linear(4,1))
model[0].bias.data[0] = -1
model[0].bias.data[1] = 1
model[0].weight.data[0] = 1
model[0].weight.data[1] = 1
model[4].weight.data[:] = 0.25
model[4].bias.data[:] = 0

x_gen = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)

fake_out = model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))
plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
display(fig)
plt.close(fig)
fig = plt.figure(figsize=(12,3))
part_model = nn.Sequential(model[0], model[1], model[2])
out_part = part_model(x_fake.unsqueeze(1))

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
display(fig)
plt.close(fig)

optim_wnorm = th.optim.Adam([model[3].weights], lr=5e-3)
optimizer = th.optim.Adam(model.parameters(), lr=5e-3)
optim_gen = th.optim.Adam([x_gen], lr=5e-3)

In [ ]:
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)

n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(x.unsqueeze(1))
    fake_out = model(x_gen.unsqueeze(1))

    loss = -(th.mean(real_out) - th.mean(fake_out))
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    x_gen.grad.data.neg_()
    optim_gen.step()
    valid_out = model(valid_x.unsqueeze(1))
    loss = -th.mean(valid_out)
    optim_wnorm.zero_grad()
    loss.backward()
    optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:
        fake_out = model(x_fake.unsqueeze(1))
        fig = plt.figure(figsize=(12,3))
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        
        display(fig)
        plt.close(fig)
        
        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_gen).squeeze(), var_to_np(x_gen).squeeze() * 0, ls='', marker='o', alpha=0.5)
        plt.xlabel('Input (generated)')
        
        display(fig)
        plt.close(fig)
        print("loss {:.3f}".format(loss.item()))
        print("wnorms", [model[3].weights])
        print(model[0].weight)
        print(model[0].bias)
        part_model = nn.Sequential(model[0], model[1], model[2])
        out_part = part_model(x_fake.unsqueeze(1))
        fig = plt.figure(figsize=(12,3))

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
        display(fig)
        plt.close(fig)

In [ ]:

part_model = nn.Sequential(model[0], model[1], model[2])
out_part = part_model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
plt.legend(("Abs first filter", # Since sum of filter after Concat ReLU
           "Abs second filter",))
display(fig)
plt.close(fig)

In [ ]:

part_model = nn.Sequential(model[0], model[1], model[2], model[3])
out_part = part_model(x_fake.unsqueeze(1))
fig = plt.figure(figsize=(12,3))

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
plt.legend(("Abs first filter", # Since sum of filter after Concat ReLU
           "Abs second filter",))
display(fig)
plt.close(fig)

### see if you dont use "weight norms"


In [ ]:
valid_x = np_to_var([-0.5, 0.5], dtype=np.float32, requires_grad=True)

from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(0, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,2), ConcatReLU(), 
                      Clip(), PointWiseMultLayer(), nn.Linear(4,1))
model[0].bias.data[0] = -1
model[0].bias.data[1] = 1
model[0].weight.data[0] = 1
model[0].weight.data[1] = 1
model[4].weight.data[:] = 0.25
model[4].bias.data[:] = 0


In [ ]:
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(x.unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss = -(th.mean(real_out) - th.mean(fake_out))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #valid_out = model(valid_x.unsqueeze(1))
    #loss = -th.mean(valid_out)
    #optim_wnorm.zero_grad()
    #loss.backward()
    #optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        display(fig)
        plt.close(fig)
        print("loss {:.3f}".format(loss.item()))
        print("wnorms", [model[3].weights])
        print(model[0].weight)
        print(model[0].bias)
        part_model = nn.Sequential(model[0], model[1], model[2])
        out_part = part_model(x_fake.unsqueeze(1))
        fig = plt.figure(figsize=(12,3))

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
        plt.legend(("Abs first filter", # Since sum of filter after Concat ReLU
                   "Abs second filter",))
        display(fig)
        plt.close(fig)

### integrate valid data into real

In [ ]:
valid_x = np_to_var([-0.5, 0.5], dtype=np.float32, requires_grad=True)

from braindecode.torch_ext.util import set_random_seeds
set_random_seeds(0, False) # worked with 102398213
model = nn.Sequential(nn.Linear(1,2), ConcatReLU(), 
                      Clip(), PointWiseMultLayer(), nn.Linear(4,1))
model[0].bias.data[0] = -1
model[0].bias.data[1] = 1
model[0].weight.data[0] = 1
model[0].weight.data[1] = 1
model[4].weight.data[:] = 0.25
model[4].bias.data[:] = 0


In [ ]:
x_fake = np_to_var(np.linspace(-2,2, 400), dtype=np.float32, requires_grad=True)


n_epochs = 2000
for i_epoch in range(n_epochs):
    real_out = model(th.cat((x, x_fake)).unsqueeze(1))
    fake_out = model(x_fake.unsqueeze(1))

    loss = -(th.mean(real_out) - th.mean(fake_out))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #valid_out = model(valid_x.unsqueeze(1))
    #loss = -th.mean(valid_out)
    #optim_wnorm.zero_grad()
    #loss.backward()
    #optim_wnorm.step()
    if i_epoch % (n_epochs // 20) == 0:

        fig = plt.figure(figsize=(12,3))
        plt.plot(var_to_np(x_fake), var_to_np(fake_out).squeeze())
        plt.ylabel('Discriminator Score')
        plt.xlabel('Input')
        display(fig)
        plt.close(fig)
        print("loss {:.3f}".format(loss.item()))
        print("wnorms", [model[3].weights])
        print(model[0].weight)
        print(model[0].bias)
        part_model = nn.Sequential(model[0], model[1], model[2])
        out_part = part_model(x_fake.unsqueeze(1))
        fig = plt.figure(figsize=(12,3))

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,0] + out_part[:,2]).squeeze())

        plt.plot(var_to_np(x_fake), var_to_np(out_part[:,1] + out_part[:,3]).squeeze())
        plt.legend(("Abs first filter", # Since sum of filter after Concat ReLU
                   "Abs second filter",))
        display(fig)
        plt.close(fig)